In [55]:
import numpy as np

X = []
Y = []

x_noise = 0.0
y_noise = 0.0

# TODO: randomly occlude large parts of the state input to simulate detection inconsistencies

model_name = "cam"

with open(f"../utils/datasets/{model_name}.csv") as f:
    for i, line in enumerate(f):
        data = line.split(",")
        x = np.array(data[2:], dtype=np.float32)
        y = np.array(data[:2], dtype=np.float32)

        x[x > 1] = 0  # track only
        y = y[1]  # steer only

        grid_size = 30
        x = x.reshape(grid_size, grid_size, 1)  # transform state back to grid

        X.append(x)
        Y.append(y)

X = np.array(X, dtype=np.float32)
Y = np.array(Y, dtype=np.float32)

print(f"Loaded {len(X)} entires")
print(f"entry {x.shape=}, {y.shape=}")

Loaded 13792 entires
entry x.shape=(30, 30, 1), y.shape=()


In [14]:
import keras
print(keras.__version__)

2.13.1


In [56]:
import keras

model = keras.models.Sequential(
    [
        # lenet-5
        keras.layers.Conv2D(6, kernel_size=(5, 5), activation="relu", input_shape=(30, 30, 1)),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(16, kernel_size=(5, 5), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dense(120, activation="relu"),
        keras.layers.Dense(84, activation="relu"),
        keras.layers.Dense(1),               
    ]
)

# TODO: add normal following to all experts' ds

model.compile(
    optimizer="adam",
    loss="mean_squared_error",
)

model.fit(
    X,
    Y,
    epochs=3,
    # validation_split=0.0,
    # callbacks=[keras.callbacks.EarlyStopping(patience=1)],
)

model.save(f"../controllers/moe_controller/models/{model_name}-new.keras")

Epoch 1/3
431/431 [==============================] - 2s 4ms/step - loss: 0.0174
Epoch 2/3
431/431 [==============================] - 2s 4ms/step - loss: 0.0096
Epoch 3/3
431/431 [==============================] - 2s 4ms/step - loss: 0.0087


In [51]:
# time inference speed
import time

start = time.time()
model(X[0:100])
end = time.time()
avg = (end - start) / 100
print(f"avg inference time: {avg*1000:.2f}ms")

avg inference time: 0.10ms
